In [5]:
"""
Personal Finance Tracker
Single-file Python CLI application.

Features:
- Add expenses (amount, category, date, description)
- List/search expenses
- Save/load expenses to/from CSV
- Generate monthly reports (text file, with totals by category and top expenses)
- Simple input validation and error handling
- Organized with an ExpenseManager class for reuse and importing

Usage:
    python personal_finance_tracker.py

Data file used by default: expenses.csv (created in the same directory)

"""

from __future__ import annotations
import csv
import os
from dataclasses import dataclass, asdict
from datetime import datetime
from typing import List, Optional, Dict, Tuple

DEFAULT_DATA_FILE = "expenses.csv"
DATE_FORMATS = ["%Y-%m-%d", "%d-%m-%Y", "%d/%m/%Y", "%Y/%m/%d", "%d %b %Y", "%d %B %Y"]


@dataclass
class Expense:
    amount: float
    category: str
    date: datetime
    description: str = ""

    def to_csv_row(self) -> List[str]:
        return [f"{self.amount:.2f}", self.category, self.date.strftime("%Y-%m-%d"), self.description]

    @staticmethod
    def from_csv_row(row: List[str]) -> Optional["Expense"]:
        try:
            amount = float(row[0])
            category = row[1]
            date = parse_date(row[2])
            description = row[3] if len(row) > 3 else ""
            if date is None:
                return None
            return Expense(amount=amount, category=category, date=date, description=description)
        except Exception:
            return None


def parse_date(text: str) -> Optional[datetime]:
    text = text.strip()
    for fmt in DATE_FORMATS:
        try:
            return datetime.strptime(text, fmt)
        except Exception:
            continue
    # try parsing ISO-like strings
    try:
        return datetime.fromisoformat(text)
    except Exception:
        return None


class ExpenseManager:
    def __init__(self, data_file: str = DEFAULT_DATA_FILE) -> None:
        self.data_file = data_file
        self.expenses: List[Expense] = []
        if os.path.exists(self.data_file):
            try:
                self.load_from_csv(self.data_file)
            except Exception:
                # if loading fails, start with empty and warn
                print(f"Warning: failed to load {self.data_file}. Starting with an empty list.")

    def add_expense(self, expense: Expense) -> None:
        self.expenses.append(expense)
        self.save_to_csv(self.data_file)

    def save_to_csv(self, path: str) -> None:
        folder = os.path.dirname(path)
        if folder and not os.path.exists(folder):
            os.makedirs(folder, exist_ok=True)
        try:
            with open(path, "w", newline='', encoding='utf-8') as f:
                writer = csv.writer(f)
                # header
                writer.writerow(["amount", "category", "date", "description"])
                for e in self.expenses:
                    writer.writerow(e.to_csv_row())
        except Exception as ex:
            print(f"Error saving to {path}: {ex}")

    def load_from_csv(self, path: str) -> None:
        loaded: List[Expense] = []
        with open(path, "r", newline='', encoding='utf-8') as f:
            reader = csv.reader(f)
            header = next(reader, None)
            for row in reader:
                if not row:
                    continue
                exp = Expense.from_csv_row(row)
                if exp:
                    loaded.append(exp)
        self.expenses = loaded

    def list_expenses(self, start: Optional[datetime] = None, end: Optional[datetime] = None, category: Optional[str] = None) -> List[Expense]:
        results = []
        for e in self.expenses:
            if start and e.date < start:
                continue
            if end and e.date > end:
                continue
            if category and e.category.lower() != category.lower():
                continue
            results.append(e)
        # sort by date
        results.sort(key=lambda x: x.date)
        return results

    def total_by_category(self, month: Optional[int] = None, year: Optional[int] = None) -> Dict[str, float]:
        totals: Dict[str, float] = {}
        for e in self.expenses:
            if month and e.date.month != month:
                continue
            if year and e.date.year != year:
                continue
            totals[e.category] = totals.get(e.category, 0.0) + e.amount
        return totals

    def total_for_month(self, month: int, year: int) -> float:
        return sum(e.amount for e in self.expenses if e.date.month == month and e.date.year == year)

    def top_expenses(self, month: Optional[int] = None, year: Optional[int] = None, n: int = 5) -> List[Expense]:
        filtered = [e for e in self.expenses if (month is None or e.date.month == month) and (year is None or e.date.year == year)]
        filtered.sort(key=lambda x: x.amount, reverse=True)
        return filtered[:n]

    def generate_monthly_report(self, month: int, year: int, out_folder: str = "reports") -> str:
        if not os.path.exists(out_folder):
            os.makedirs(out_folder, exist_ok=True)
        filename = f"report_{year}_{month:02d}.txt"
        path = os.path.join(out_folder, filename)

        totals = self.total_by_category(month=month, year=year)
        total = self.total_for_month(month, year)
        top = self.top_expenses(month=month, year=year, n=10)
        month_name = datetime(year, month, 1).strftime("%B %Y")

        lines = []
        lines.append(f"Personal Finance Report — {month_name}")
        lines.append("=" * 40)
        lines.append(f"Total spending: {total:.2f}")
        lines.append("")
        lines.append("Spending by category:")
        if totals:
            for cat, amt in sorted(totals.items(), key=lambda x: x[1], reverse=True):
                lines.append(f"  {cat}: {amt:.2f}")
        else:
            lines.append("  No expenses recorded for this month.")
        lines.append("")
        lines.append("Top expenses:")
        if top:
            for e in top:
                lines.append(f"  {e.date.strftime('%Y-%m-%d')}: {e.category} — {e.amount:.2f} ({e.description})")
        else:
            lines.append("  No expenses recorded for this month.")

        # Save report
        try:
            with open(path, "w", encoding='utf-8') as f:
                f.write("\n".join(lines))
        except Exception as ex:
            raise RuntimeError(f"Failed to write report: {ex}")

        return path


# ------ CLI Helpers ------

def prompt_date(prompt: str = "Date (YYYY-MM-DD): ") -> datetime:
    while True:
        raw = input(prompt).strip()
        dt = parse_date(raw)
        if dt:
            return dt
        print("Invalid date. Try formats like 2025-11-14 or 14-11-2025 or 14/11/2025.")

def prompt_amount(prompt: str = "Amount: ") -> float:
    while True:
        raw = input(prompt).strip()
        try:
            amt = float(raw)
            if amt < 0:
                print("Amount cannot be negative.")
                continue
            return amt
        except ValueError:
            print("Enter a valid number, e.g. 12.50")

def prompt_nonempty(prompt: str) -> str:
    while True:
        val = input(prompt).strip()
        if val:
            return val
        print("Value cannot be empty.")

def print_expenses(expenses: List[Expense]) -> None:
    if not expenses:
        print("No expenses found.")
        return
    print(f"{'Date':10}  {'Amount':10}  {'Category':15}  Description")
    print("-" * 60)
    for e in expenses:
        date_s = e.date.strftime("%Y-%m-%d")
        print(f"{date_s:10}  {e.amount:10.2f}  {e.category:15}  {e.description}")

def main_menu(manager: ExpenseManager) -> None:
    MENU = """
Personal Finance Tracker
-----------------------
1) Add expense
2) List all expenses
3) List expenses by month
4) List expenses by category
5) Generate monthly report
6) Import expenses from another CSV
7) Export filtered CSV
8) Exit
"""
    while True:
        print(MENU)
        choice = input("Choose an option (1-8): ").strip()
        if choice == "1":
            amt = prompt_amount()
            cat = prompt_nonempty("Category (e.g., Groceries, Rent, Transport): ")
            date = prompt_date()
            desc = input("Description (optional): ").strip()
            exp = Expense(amount=amt, category=cat, date=date, description=desc)
            manager.add_expense(exp)
            print("Expense added and saved.")

        elif choice == "2":
            exps = manager.list_expenses()
            print_expenses(exps)

        elif choice == "3":
            year = int(prompt_nonempty("Year (e.g., 2025): "))
            month = int(prompt_nonempty("Month (1-12): "))
            start = datetime(year, month, 1)
            # compute end as last day of month
            if month == 12:
                end = datetime(year + 1, 1, 1)
            else:
                end = datetime(year, month + 1, 1)
            exps = manager.list_expenses(start=start, end=end)
            print_expenses(exps)

        elif choice == "4":
            cat = prompt_nonempty("Category to filter by: ")
            exps = manager.list_expenses(category=cat)
            print_expenses(exps)

        elif choice == "5":
            year = int(prompt_nonempty("Year (e.g., 2025): "))
            month = int(prompt_nonempty("Month (1-12): "))
            try:
                path = manager.generate_monthly_report(month=month, year=year)
                print(f"Report generated: {path}")
            except Exception as ex:
                print(f"Failed to generate report: {ex}")

        elif choice == "6":
            path = prompt_nonempty("Path to CSV to import: ")
            if not os.path.exists(path):
                print("File not found.")
            else:
                try:
                    manager.load_from_csv(path)
                    # Save to default file after import
                    manager.save_to_csv(manager.data_file)
                    print(f"Imported and saved to {manager.data_file}")
                except Exception as ex:
                    print(f"Import failed: {ex}")

        elif choice == "7":
            # Export filtered CSV by month/year or category
            filter_type = input("Filter by (none/month/category): ").strip().lower()
            filtered = manager.expenses
            if filter_type == "month":
                year = int(prompt_nonempty("Year (e.g., 2025): "))
                month = int(prompt_nonempty("Month (1-12): "))
                filtered = [e for e in manager.expenses if e.date.month == month and e.date.year == year]
            elif filter_type == "category":
                cat = prompt_nonempty("Category: ")
                filtered = [e for e in manager.expenses if e.category.lower() == cat.lower()]
            out = prompt_nonempty("Output CSV path (e.g., out.csv): ")
            try:
                with open(out, "w", newline='', encoding='utf-8') as f:
                    writer = csv.writer(f)
                    writer.writerow(["amount", "category", "date", "description"])
                    for e in filtered:
                        writer.writerow(e.to_csv_row())
                print(f"Exported {len(filtered)} rows to {out}")
            except Exception as ex:
                print(f"Failed to export: {ex}")

        elif choice == "8":
            print("Goodbye!")
            break
        else:
            print("Invalid choice. Enter a number between 1 and 8.")

if __name__ == "__main__":
    print("Starting Personal Finance Tracker...")
    manager = ExpenseManager()
    try:
        main_menu(manager)
    except KeyboardInterrupt:
        print("\nExiting. Goodbye.")


Starting Personal Finance Tracker...

Personal Finance Tracker
-----------------------
1) Add expense
2) List all expenses
3) List expenses by month
4) List expenses by category
5) Generate monthly report
6) Import expenses from another CSV
7) Export filtered CSV
8) Exit



Choose an option (1-8):  List all expenses


Invalid choice. Enter a number between 1 and 8.

Personal Finance Tracker
-----------------------
1) Add expense
2) List all expenses
3) List expenses by month
4) List expenses by category
5) Generate monthly report
6) Import expenses from another CSV
7) Export filtered CSV
8) Exit



Choose an option (1-8):  5
Year (e.g., 2025):  2025
Month (1-12):  12


Report generated: reports\report_2025_12.txt

Personal Finance Tracker
-----------------------
1) Add expense
2) List all expenses
3) List expenses by month
4) List expenses by category
5) Generate monthly report
6) Import expenses from another CSV
7) Export filtered CSV
8) Exit



Choose an option (1-8):  8


Goodbye!
